## ATHENA: Protein-Level IDP Classification Tutorial
This Google Colab notebook provides a step-by-step guide for running the ATHENA protein-level classifier (ATHENA_IDP_classification.py).

This tutorial will guide you through cloning the repository, setting up the environment, preparing data, and running the prediction pipeline to generate the ATHENA Score for your protein sequences.

### Expected Runtime

**Anywhere from 1-10 minutes**, depending on the number of sequences to be scored.

### Step 1: Clone Repository and Set Up Environment
First, we will clone the ATHENA GitHub repository and move into the new directory. Then, we'll install the required Python libraries.

In [1]:
# Colab Cell 1: Clone Repo & Install Dependencies

# Clone the Repository
!git clone https://github.com/Shimizu-team/ATHENA
%cd ATHENA

# Install Dependencies
!pip install torch transformers peft tqdm einops

print("\n Repository cloned and dependencies installed.")

Cloning into 'ATHENA'...
remote: Repository not found.
fatal: repository 'https://github.com/YOUR_USERNAME/ATHENA/' not found
[Errno 2] No such file or directory: 'ATHENA'
/Users/koheiota/Desktop/251028_ATHENA
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.8/150.8 MB 10.3 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 14.6 MB/s eta 0:00:00 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 566.1/566.1 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 20.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 22.2 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 18.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.2/536.2 kB 11.1 MB/s eta 0:00:00

✅ Repository cloned and dependencies installed.


### Step 2: Import Required Libraries
Now that we are in the repository's directory, we can import the necessary modules from your scripts, along with standard libraries.

In [1]:
# Import Libraries
import os
import sys
import torch
import pandas as pd
import argparse
import json
from google.colab import files

from config import Config
from ATHENA_IDP_classification import predict


/Users/koheiota/anaconda3/envs/Tardigrada/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


✅ Core modules imported successfully.


### Step 3: Prepare Model Parameters and Input Data
This step involves preparing all the necessary files for the model to run.

**3. Unzip Model Parameters**

Model parameters are in a zip file. The script expects the unzipped files to be in a directory named ATHENA_IDP_model_params.

In [5]:
# Define the directory and the zip file
model_dir = "ATHENA_IDP_model_params"
zip_filename = "ATHENA_IDP_model_params.zip"
zip_filepath = os.path.join(model_dir, zip_filename)

print(f"Attempting to unzip '{zip_filepath}' into its parent directory '{model_dir}'...")

# Check if the zip file exists at the specified path
if not os.path.isfile(zip_filepath):
    print(f"\n⚠️ Error: File not found at '{zip_filepath}'.")
    print("Please ensure the GitHub repository was cloned correctly and this file exists.")
else:
    print(f"\nExtracting '{zip_filepath}'...")
    
    !unzip -q -o "{zip_filepath}" -d "{model_dir}"
    
    print(f"Unzip command finished. Files extracted to '{model_dir}'.")
    

Attempting to unzip 'ATHENA_IDP_model_params/ATHENA_IDP_model_params.zip' into its parent directory 'ATHENA_IDP_model_params'...

Extracting 'ATHENA_IDP_model_params/ATHENA_IDP_model_params.zip'...
✅ Unzip command finished. Files extracted to 'ATHENA_IDP_model_params'.

--- New contents of 'ATHENA_IDP_model_params' ---
total 120736
-rw-r--r--  1 koheiota  staff  29404325 Oct 28 10:31 ATHENA_IDP_model_params.zip
-rwxr-xr-x@ 1 koheiota  staff       805 Oct 28 10:29 adapter_config.json
-rwxr-xr-x@ 1 koheiota  staff  31420456 Oct 28 10:29 adapter_model.safetensors
-rwxr-xr-x@ 1 koheiota  staff     17044 Oct 28 10:29 classifier_params.pth
-rwxr-xr-x@ 1 koheiota  staff     15024 Oct 28 10:29 rng_state_0.pth
-rwxr-xr-x@ 1 koheiota  staff     15024 Oct 28 10:29 rng_state_1.pth
-rwxr-xr-x@ 1 koheiota  staff     15024 Oct 28 10:29 rng_state_2.pth
-rwxr-xr-x@ 1 koheiota  staff     15024 Oct 28 10:29 rng_state_3.pth
-rwxr-xr-x@ 1 koheiota  staff      1064 Oct 28 10:29 scheduler.pt
-rwxr-xr-x@ 1 ko

### Step 4: Define Run Configuration

In [12]:
# Colab Cell 4: Define Configuration
os.makedirs("output", exist_ok=True)
manual_args = argparse.Namespace()

# Basic Settings
manual_args.output_type = "before_softmax" # "IDP_probability" or "before_softmax"
manual_args.base_model = "Synthyra/ESMplusplus_small"
manual_args.classifier_params_path = "ATHENA_IDP_model_params" # Relative path to unzipped folder
manual_args.fasta_path = "input/example_sequences.fasta" # Relative path to created file
manual_args.output_dir = "output"

# Adapter Settings
# Simulates: --adapter_paths "IDP_LoRA=model_params"
manual_args.adapter_paths = {"IDP_LoRA": "ATHENA_IDP_model_params"} 
manual_args.adapter_weights = None # Not needed for a single adapter

# Model Settings 
manual_args.num_labels = 2
manual_args.max_length = 10000
manual_args.batch_size = 32 # Adjust based on Colab GPU memory (e.g., T4)

# Output Settings
manual_args.title = "IDP_Inference_Colab" # Title for output files

# Create the final config object
conf = Config(manual_args)

print("--- Running Configuration ---")
print(json.dumps(vars(conf), indent=2))
print("-----------------------------")

--- Running Configuration ---
{
  "output_type": "before_softmax",
  "base_model": "Synthyra/ESMplusplus_small",
  "classifier_params_path": "ATHENA_IDP_model_params",
  "fasta_path": "input/example_sequences.fasta",
  "output_dir": "output",
  "adapter_paths": {
    "IDP_LoRA": "ATHENA_IDP_model_params"
  },
  "adapter_weights": null,
  "num_labels": 2,
  "max_length": 10000,
  "batch_size": 32,
  "title": "IDP_Inference_Colab"
}
-----------------------------


### Step 5: Run Prediction 
With all scripts imported, models loaded, and configuration set, we can now call the predict function.

This will:

1. Download the base model (Synthyra/ESMplusplus_small) from Hugging Face.

2. Load the base model and apply your LoRA adapter from model_params.

3. Load the fine-tuned classifier_params.pth.

4. Process the sequences from example_sequences.fasta in batches.

5. Save the results to the output directory.

In [13]:
import torch

print("--- Starting Prediction ---")
try:
    # Call the main predict function from your script
    predictions = predict(conf)
    print("\nPrediction Complete")

except FileNotFoundError as e:
    print(f"\nExecution Error")
    print(e)
    print("Prediction failed. Please double-check that your model files (e.g., 'classifier_params.pth')")
    print("are correctly located inside the 'model_params' directory after unzipping.")

--- Starting Prediction ---
Using device: cpu
Using device: cpu


Some weights of ESMplusplusForSequenceClassification were not initialized from the model checkpoint at Synthyra/ESMplusplus_small and are newly initialized: ['classifier.0.bias', 'classifier.0.weight', 'classifier.2.bias', 'classifier.2.weight', 'classifier.3.bias', 'classifier.3.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Loading single LoRA adapter 'IDP_LoRA'...
Single LoRA adapter 'IDP_LoRA' loaded successfully.

Model setup complete.
Loaded classifier weights from ATHENA_IDP_model_params/classifier_params.pth
Model is ready on cpu
> 3 sequences loaded from input/example_sequences.fasta
Starting prediction for 3 sequences with batch size 32...


Predicting in batches:   0%|          | 0/1 [04:01<?, ?it/s]


: 

: 

### Step 6: Review and Interpret Results
The predict function saves the results as a .pt file, but also prints a summary. Let's load the saved file using torch and pandas for a cleaner view.

Since we set --output_type "before_softmax", the output is the raw logit score (the "ATHENA Score").

In [8]:
import torch
import pandas as pd
import os

# Determine the output file path based on our config
if conf.output_type == "before_softmax":
    output_file = os.path.join(conf.output_dir, f"IDP_score_before_softmax_{conf.title}.pt")
    column_name = "ATHENA Score (Logit)"
else:
    output_file = os.path.join(conf.output_dir, f"IDP_score_{conf.title}.pt")
    column_name = "IDP Probability"

# Check if the file was created
if os.path.exists(output_file):
    print(f"Loading results from '{output_file}'...")
    
    # Load the saved dictionary (mapping to CPU for safety)
    data = torch.load(output_file, map_location='cpu')
    
    # Convert to Pandas DataFrame for nice formatting
    df = pd.DataFrame(list(data.items()), columns=['Sequence ID', column_name])
    
    # Sort by score, descending
    df_sorted = df.sort_values(by=column_name, ascending=False).reset_index(drop=True)
    
    print(f"\nPrediction Results (Top {len(df_sorted)})")
    
    # Display as a clean markdown table
    print(df_sorted.to_markdown(index=False))

else:
    print(f"Output file '{output_file}' not found.")
    print("Please ensure Step 5 completed without errors.")

Note: you may need to restart the kernel to use updated packages.
